## Bayesian Logistic Ridge Regression
Script Name: Model_Analysis.ipynb

Author: Brian Cain


This notebook takes the cleaned and feature engineering data from modelData.csv and uses it to create a Bayesian Logistic Ridge Regression model. It then analyzes the results of this model using posterior predictive distributions and comparing its performance to frequentist bayesian logistic regression. 

<hr>

In [1]:
##Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##Pull in and display the data
data = pd.read_csv('D:\\College_Football_Model_Data\\modelData.csv')
data.head()

,gameId,school,week_num,gameSeason,win,offDiff_rush_td_movAvg,defDiff_pass_td_movAvg,offDiff_yp_rush_movAvg,defDiff_pass_yards_movAvg,offDiff_total_yards_movAvg,defDiff_turnovers_movAvg,defDiff_firstDowns_movAvg,defDiff_offensive_ppa_movAvg,offDiff_offensive_powerSuccess_movAvg,defDiff_completion_pct_movAvg,defDiff_third_pct_movAvg,gameDiff_gameControl_movAvg,gameDiff_elo_movAvg
0,400764869,Temple,3,2015.0,1.0,-1.333333,-0.333333,-0.433333,-13.833333,-151.000000,0.666667,1.000000,-0.047219,-0.089744,-0.034619,0.046491,6.750000,324.833333
1,400763604,UTEP,3,2015.0,1.0,-1.333333,1.333333,-1.133333,57.333333,-190.000000,-0.666667,4.333333,0.346266,0.111111,0.055580,0.150253,-2.833333,225.666667
2,400756922,Georgia Tech,3,2015.0,0.0,2.500000,-1.333333,2.566667,-44.500000,148.666667,1.333333,-4.833333,-0.166122,0.444444,-0.098090,0.070130,1.708333,172.333333
3,400603852,South Carolina,3,2015.0,0.0,0.666667,-0.666667,2.233333,30.333333,53.333333,1.000000,0.333333,-0.097716,-0.022222,0.034252,-0.064306,-9.833333,-240.333333
4,400603853,Kentucky,3,2015.0,0.0,1.333333,-1.666667,3.300000,-45.000000,100.333333,0.666667,1.333333,-0.124125,-0.111111,-0.050671,-0.135313,-4.500000,-290.333333


In [2]:
##Perform z-score normalization of the data
def z_score_normalization(df,col):
    
    ##Compute variables necessary for this 
    mu = np.mean(df[col])
    std = np.std(df[col])
    
    ##Replace column with normalized column
    df[col] = (df[col]-mu)/std
    
    return df

##Now perform normalization
predictors = ['offDiff_rush_td_movAvg', 'defDiff_pass_td_movAvg','offDiff_yp_rush_movAvg', 
              'defDiff_pass_yards_movAvg','offDiff_total_yards_movAvg', 'defDiff_turnovers_movAvg',
              'defDiff_firstDowns_movAvg', 'defDiff_offensive_ppa_movAvg',
              'offDiff_offensive_powerSuccess_movAvg','defDiff_completion_pct_movAvg',
              'defDiff_third_pct_movAvg','gameDiff_gameControl_movAvg','gameDiff_elo_movAvg']
for i in predictors:
    data = z_score_normalization(data,i)
data.head()


,gameId,school,week_num,gameSeason,win,offDiff_rush_td_movAvg,defDiff_pass_td_movAvg,offDiff_yp_rush_movAvg,defDiff_pass_yards_movAvg,offDiff_total_yards_movAvg,defDiff_turnovers_movAvg,defDiff_firstDowns_movAvg,defDiff_offensive_ppa_movAvg,offDiff_offensive_powerSuccess_movAvg,defDiff_completion_pct_movAvg,defDiff_third_pct_movAvg,gameDiff_gameControl_movAvg,gameDiff_elo_movAvg
0,400764869,Temple,3,2015.0,1.0,-1.371113,-0.399361,-0.374039,-0.262128,-1.743327,0.938524,0.203776,-0.354374,-0.651392,-0.444200,0.500301,1.375274,1.300499
1,400763604,UTEP,3,2015.0,1.0,-1.371113,1.441503,-0.947174,0.798175,-2.181374,-0.964484,1.017180,2.496855,0.710578,0.710779,1.656307,-0.440968,0.947893
2,400756922,Georgia Tech,3,2015.0,0.0,2.502526,-1.503880,2.082252,-0.719026,1.622522,1.890027,-1.219681,-1.215956,2.970868,-1.256929,0.763657,0.419773,0.758256
3,400603852,South Carolina,3,2015.0,0.0,0.649916,-0.767534,1.809331,0.395906,0.551740,1.414276,0.041095,-0.720285,-0.193538,0.437671,-0.734100,-1.767614,-0.709059
4,400603853,Kentucky,3,2015.0,0.0,1.323592,-1.872053,2.682679,-0.726476,1.079643,0.938524,0.285116,-0.911648,-0.796282,-0.649748,-1.525188,-0.756836,-0.886843


### Defining the Log-Posterior

In [22]:
##Define function to compute log-likelihood
def logLike(y,x,betas):
    
    ##Compute eta
    eta = np.matmul(x,betas)
    
    ##Compute sum 1 (sum(y_i * eta_i))
    sum1 = sum(y*eta)
    
    ##Compute sum 2 (sum(ln(1+e^(eta_i))))
    sum2 = sum(np.log((1+np.exp(eta))))
    
    ##Compute the final likelihood
    computed_logLike = sum1 - sum2
    
    return computed_logLike

##Define function to compute log prior for betas
def beta_logPrior(betas,lambda_):
    
    computed_betaPrior = sum(np.log((lambda_**(.5))*np.exp(((betas*betas)*-lambda_))))
    
    return computed_betaPrior

##Define function to compute log prior of lambda (shrinkage parameter)
def lambda_logPrior(lambda_):
    
    computed_lambdaPrior = np.log(np.exp(-1.5*lambda_))
    
    return computed_lambdaPrior

##Define function that computes the log posterior
def logPosterior(y,x,betas,lambda_):
    
    ##Add up log likelihood, and the priors
    computed_logPost = logLike(y,x,betas) + beta_logPrior(betas,lambda_) + lambda_logPrior(lambda_)
    
    return computed_logPost

### Metropolis-Hastings Algorithm

<b>Proposal Distribution:</b>

Frequentist comparison: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html